In [1]:
import numpy as np
import pandas as pd

In [2]:
class WRMSSEEvaluator:
    def __init__(self, sales_df, calendar_df, prices_df, weight_cache=None):
        # sales_df: wide (rows=series, cols=d_1..d_T), plus id keys
        self.sales_df = sales_df
        self.calendar = calendar_df
        self.prices = prices_df
        self.weight_cache = weight_cache
        self.levels = [
            ["all"], ["state_id"], ["store_id"], ["cat_id"], ["dept_id"],
            ["state_id","cat_id"], ["state_id","dept_id"],
            ["store_id","cat_id"], ["store_id","dept_id"],
            ["item_id"], ["state_id","item_id"], ["item_id","store_id"]
        ]  # 12 levels [2]

    def _build_aggregates(self, df_keys_vals):
        # df_keys_vals: keys + d_ cols
        aggs = []
        for keys in self.levels:
            if keys == ["all"]:
                agg = df_keys_vals.filter(like="d_").sum(axis=0).to_frame().T
                agg["all"] = "Total"
            else:
                agg = df_keys_vals.groupby(keys).sum(numeric_only=True).reset_index()
            aggs.append((keys, agg))
        return aggs  # list of (keys, df) [2]

    def _scale_denominators(self, train_df):
        # per series: mean squared diff over nonzero trimmed history
        y = train_df.filter(like="d_").values
        den = np.array([np.mean(np.diff(np.trim_zeros(row))**2) for row in y])
        den[~np.isfinite(den)] = np.nan
        return den  # length = n_series [2]

    def _weights(self, sales_train_end_week):
        # compute revenue weights using last 28 days prices * sales at each level
        # join sell_prices on wm_yr_wk, then aggregate revenue shares
        # return aligned weights per series across all 12 levels
        # Placeholder signature; see notebook refs for full detail. [2][1]
        pass

    def score(self, y_true_df, y_pred_df, train_df):
        # y_true_df, y_pred_df: same index/keys as sales_df and 28 d_ cols
        # Compute per-level RMSSE then weighted sum
        results = []
        for keys in self.levels:
            # align frames at level
            if keys == ["all"]:
                yt = y_true_df.filter(like="d_").sum(axis=0).to_frame().T
                yp = y_pred_df.filter(like="d_").sum(axis=0).to_frame().T
                tr = train_df.filter(like="d_").sum(axis=0).to_frame().T
                den = np.mean(np.diff(tr.values.squeeze())**2)  # scalar
                rmsse = np.sqrt(np.mean((yt.values - yp.values)**2) / den)
                w = 1.0  # replace with revenue-based weight
                results.append(w * rmsse)
            else:
                yt = y_true_df.groupby(keys).sum(numeric_only=True)
                yp = y_pred_df.groupby(keys).sum(numeric_only=True).reindex(yt.index)
                tr = train_df.groupby(keys).sum(numeric_only=True).reindex(yt.index)
                den = np.array([np.mean(np.diff(r)**2) for r in tr.filter(like="d_").values])
                num = np.mean((yt.filter(like="d_").values - yp.filter(like="d_").values)**2, axis=1)
                rmsse = np.sqrt(num / den)
                w = np.ones_like(rmsse)  # replace with revenue-based weights
                results.append(np.nansum(w * rmsse) / np.nansum(w))
        return float(np.mean(results))

In [3]:
def build_features(df, max_window=900):
    # df: long or wide converted to long per SKU-store with columns:
    # ['id','item_id','dept_id','cat_id','store_id','state_id','date','d','sales','price',
    #  'dow','month','snap','event_type', ...]
    df = df.sort_values(['item_id','store_id','date'])
    # target lags
    for L in [1, 7, 28]:
        df[f"lag_{L}"] = df.groupby(['item_id','store_id'])['sales'].shift(L)
    # rolling on lag_28 to prevent leakage
    grp = df.groupby(['item_id','store_id'])
    for W in [7, 28, 56]:
        df[f"rmean_{W}"] = grp['lag_28'].transform(lambda s: s.rolling(W, min_periods=1).mean())
        df[f"rstd_{W}"]  = grp['lag_28'].transform(lambda s: s.rolling(W, min_periods=1).std())
    # price features
    df['log_price'] = np.log(df['price'].replace(0, np.nan)).fillna(method='ffill')
    df['price_rolmin_8w'] = grp['price'].transform(lambda s: s.rolling(8, min_periods=1).min())
    df['promo_flag'] = (df['price'] <= 1.05*df['price_rolmin_8w']).astype(np.int8)
    # compact calendar
    df['dow_cat'] = df['dow'].astype('int8')
    df['month_cat'] = df['month'].astype('int8')
    # drop rows with insufficient history for lags
    return df.dropna(subset=['lag_28']).reset_index(drop=True)

In [4]:
import lightgbm as lgb
import numpy as np

BASE_PARAMS = dict(
    objective="poisson",  # count-like targets; consider 'regression' if negative vals appear
    learning_rate=0.05,
    num_leaves=31,
    max_depth=6,
    min_data_in_leaf=50,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=5000,
    reg_alpha=0.1,
    reg_lambda=0.1,
    verbosity=-1
)

CAT_COLS = ['item_id','dept_id','cat_id','store_id','state_id','dow_cat','month_cat']

def train_per_horizon(train_df, valid_df, feature_cols, target_cols):
    boosters = []
    for h, tgt in enumerate(target_cols, start=1):  # e.g., y_t+1 ... y_t+28 columns
        dtrain = lgb.Dataset(train_df[feature_cols], label=train_df[tgt], categorical_feature=CAT_COLS, free_raw_data=False)
        dvalid = lgb.Dataset(valid_df[feature_cols], label=valid_df[tgt], categorical_feature=CAT_COLS, free_raw_data=False)
        booster = lgb.train(
            params=BASE_PARAMS,
            train_set=dtrain,
            valid_sets=[dvalid],
            valid_names=[f"val_h{h}"],
            num_boost_round=5000,
            early_stopping_rounds=200,
            verbose_eval=False
        )
        boosters.append(booster)
    return boosters

def predict_per_horizon(models, df, feature_cols):
    preds = []
    for booster in models:
        preds.append(booster.predict(df[feature_cols], num_iteration=booster.best_iteration))
    return np.stack(preds, axis=1)

In [6]:
Q_PARAMS = dict(objective="quantile", learning_rate=0.05, num_leaves=31, max_depth=6,
                n_estimators=3000, subsample=0.8, colsample_bytree=0.8, min_data_in_leaf=50)

def train_quantiles(X_tr, y_tr, X_va, y_va, cat_cols, alphas=(0.1,0.5,0.9)):
    models = {}
    for a in alphas:
        mdl = lgb.LGBMRegressor(**Q_PARAMS, alpha=a)
        mdl.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], categorical_feature=cat_cols,
                eval_metric="quantile", verbose=False)
        models[a] = mdl
    return models

def predict_quantiles(models, X):
    preds = {a: m.predict(X) for a, m in models.items()}
    # enforce monotone order
    import numpy as np
    p10, p50, p90 = preds[0.1], preds[0.5], preds[0.9]
    p50 = np.maximum(np.minimum(p50, p90), p10)
    p10 = np.minimum(p10, p50); p90 = np.maximum(p90, p50)
    return p10, p50, p90
# bootstrap_pi.py
import numpy as np

def bootstrap_intervals(point_forecasts, residuals, quantiles=(0.1,0.9), n_boot=200, seed=42):
    rng = np.random.default_rng(seed)
    B, H = n_boot, point_forecasts.shape[21]
    draws = rng.choice(residuals, size=(B, H), replace=True)
    sims = point_forecasts[np.newaxis, :, :] + draws[:, np.newaxis, :]  # broadcast per series if needed
    lower = np.quantile(sims, quantiles, axis=0)
    upper = np.quantile(sims, quantiles[21], axis=0)
    return lower, upper

In [7]:
import numpy as np

def toggle_promo(features, on=True):
    feat = features.copy()
    feat['promo_flag'] = np.where(on, 1, 0).astype(np.int8)
    return feat

def toggle_snap(features, state_col='state_id', target_state='CA', on=True):
    feat = features.copy()
    mask = feat[state_col] == target_state
    feat.loc[mask, 'snap'] = 1 if on else 0
    return feat

def shift_price(features, pct=-0.10):
    feat = features.copy()
    feat['log_price'] = np.log(np.exp(feat['log_price']) * (1.0 + pct))
    # recompute promo if available
    if 'price_rolmin_8w' in feat:
        price = np.exp(feat['log_price'])
        feat['promo_flag'] = (price <= 1.05*feat['price_rolmin_8w']).astype(np.int8)
    return feat

def score_scenario(model, feat_baseline, transform_fn):
    X0 = feat_baseline.copy()
    y0 = model.predict(X0)
    X1 = transform_fn(X0)
    y1 = model.predict(X1)
    return y1 - y0